# (1) Detect Human

This function takes a path to an image as input & returns "True" if there's a detected face.  The plan is to replace this Python implementation with the iOS implementation [here](https://developer.apple.com/documentation/vision/vndetectfacerectanglesrequest).

### notes on iOS

- Seems straightforward to use the documentation to accomplish this functionality! 

In [1]:
def face_detector(img_path):
    """ returns "True" if face is detected in image stored at img_path """
    import cv2
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

# (2) Detect Dog

The ResNet-50 model takes a pre-processed image as input and outputs the probability that the image contains an object from each of the 1000 categories here.

The `path_to_tensor` function preprocesses the image.  In Python/Keras, this is a necessary step before the model will accept the image.

The `dog_detector` function takes the preprocessed image as input.  It then supplies the pre-processed image to ResNet-50, which returns a (probability) vector with 1000 entries (where the $i$-th entry in the vector is a number between 0 and 1 that yields the probability that the image depicts the $i$-th object category).  The categories that are different dog types should correspond to entries 151-268 in the vector, inclusive.

### notes on iOS

- We need to preprocess the image:
    - Subtract a fixed pixel from every pixel in the image.  If the pixel is expressed in RGB, with red, green, and blue intensities taking on values $[0,255]$, then we need to subtract $[103.939, 116.779, 123.68]$ from every pixel.
    - Convert the image from RGB to BGR.
    - Resize image to 224 x 224 (In Keras, the image needs to be reshaped to a tensor with dimensions `1 x 224 x 224 x 3`, before ResNet-50 will accept the image.  Here, `3` corresponds to the number of color channels (BGR) ... But it looks like for iOS, the image just needs to be a color image that's 224 x 224.)
- In my understanding, the Core ML-compatible ResNet-50 model that is supplied **in the documentation** should return a 1000-dimensional probability vector.  In this case, the dog detector function should be straightforward to implement:
    - We need only take the arg max of that probability vector, which yields the predicted object category (... the arg max will be a value between 0 and 999, inclusive).  
    - The image is deemed by the model to contain a dog if the arg max lies between 151 and 268, inclusive.  Here's the full list of categories: [link](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

In [2]:
def path_to_tensor(img_path):
    """ preprocessing - resize the image, subtract mean pixel, change dimensionality of input """
    from keras.preprocessing import image   
    import numpy as np
    from keras.applications.resnet50 import preprocess_input
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return preprocess_input(x)

def dog_detector(proc_tensor):
    """ returns "True" if a dog is detected in the image stored at img_path """
    from keras.applications.resnet50 import preprocess_input, ResNet50
    import numpy as np
    ResNet50_model = ResNet50(weights='imagenet')
    prediction = np.argmax(ResNet50_model.predict(proc_tensor))
    return ((prediction <= 268) & (prediction >= 151))

# (3) Get Breed

It will take some extra effort to make the student model work (we should talk about this by video, I think).  For now, I think that the best bet is to get a pipeline working that uses just the ResNet-50 model for predicting dog breed.  This model already exists in the iOS documentation ...

The idea would be:
- if a human is detected in Step 1, return the dog that it is predicted to look most like.  
- if a dog is detected in Step 2, return the corresponding breed of dog

... I think that this first attempt would be easily replaced later with the student's model.

### notes on iOS

For now, I think that it's possible to simply return the most predicted dog breed from the ResNet-50 model that exists in the iOS documentation ...

In [3]:
def ResNet50_model_predict_breed(proc_tensor):
    """ uses the ResNet-50 model from the documentation to predict dog breed """
    from keras.applications.resnet50 import ResNet50, decode_predictions
    ResNet50_model = ResNet50(weights='imagenet')
    return decode_predictions(ResNet50_model.predict(proc_tensor), top=0)[0][0][1]

# Full Pipeline: (1) + (2) + (3) (w/ ResNet-50)

In [4]:
def full_pipeline(img_path):
    import matplotlib.pyplot as plt
    %matplotlib inline
    # get the tensor that will be used to predict breed
    proc_tensor = path_to_tensor(img_path)
    # print the appropriate statements
    if face_detector(img_path):
        print('hello, human! you vaguely resemble ...')
    elif dog_detector(proc_tensor):
        print('hello, dog! you look like a ...')
    else:
        print('hmm, neither human nor dog! but it reminds me of ...')
    print(ResNet50_model_predict_breed(proc_tensor))

# And that's it!

In [5]:
full_pipeline('test_images/human.jpg')

Using TensorFlow backend.


hello, human! you vaguely resemble ...
stole


In [6]:
full_pipeline('test_images/dog.jpg')

hello, dog!  you look like a ...
Boston_bull


# Saving the ResNet-50 Model for iOS (as HDF5 file)

In [7]:
# from keras.applications.resnet50 import ResNet50
# model = ResNet50(weights='imagenet')
# model.save('ResNet50_for_iOS.h5')

# Saving the Student's Model for iOS (as HDF5 file)

In [19]:
from keras.applications.resnet50 import ResNet50
from keras.models import load_model, Model
from keras.layers import GlobalAveragePooling2D, Dropout, Dense

resnet_50_model = ResNet50(weights='imagenet', include_top=False)
last = resnet_50_model.output

x = GlobalAveragePooling2D()(last)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
preds = Dense(133, activation='softmax')(x)

student_model = Model(resnet_50_model.input, preds)

# print layers in student_model
print('layers in student_model')
for layer in student_model.layers[-4:]:
    print(layer)

# print layers in student_model_stump
print('\n layers in student_model_stump')
student_model_stump = load_model('student_model_stump.h5')
for layer in student_model_stump.layers:
    print(layer)
    
student_model_stump.summary()

layers in student_model

 layers in student_model_stump
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               68229     
Total params: 1,117,317
Trainable params: 1,117,317
Non-trainable params: 0
_________________________________________________________________


In [20]:
student_model.layers[-4].set_weights(student_model_stump.layers[0].get_weights())
student_model.layers[-3].set_weights(student_model_stump.layers[1].get_weights())
student_model.layers[-2].set_weights(student_model_stump.layers[2].get_weights())
student_model.layers[-1].set_weights(student_model_stump.layers[3].get_weights())

In [21]:
student_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# student_model.save('student_model_for_iOS.h5')

# Full Pipeline: (1) + (2) + (3) (w/ student model)

In [11]:
# for writing dog names into txt file
'''
from glob import glob
dog_names = [item[58:-1] for item in sorted(glob("../../CNN content/dog-project-private/dogImages/train/*/"))]
print(dog_names)

thefile = open('dog_names.txt', 'w')
for item in dog_names:
    thefile.write("%s\n" % item)
'''

'\nfrom glob import glob\ndog_names = [item[58:-1] for item in sorted(glob("../../CNN content/dog-project-private/dogImages/train/*/"))]\nprint(dog_names)\n\nthefile = open(\'dog_names.txt\', \'w\')\nfor item in dog_names:\n    thefile.write("%s\n" % item)\n'

In [15]:
def student_model_predict_breed(proc_tensor):
    """ uses the student's model to predict dog breed """
    from keras.models import load_model
    import numpy as np
    student_model = load_model('student_model.h5')
    text_file = open('dog_names.txt', 'r')
    dog_names = text_file.read().split('\n')
    return dog_names[np.argmax(student_model.predict(proc_tensor))]

In [16]:
def full_pipeline_student(img_path):
    import matplotlib.pyplot as plt
    %matplotlib inline
    # get the tensor that will be used to predict breed
    proc_tensor = path_to_tensor(img_path)
    # print the appropriate statements
    if face_detector(img_path):
        print('hello, human! you vaguely resemble ...')
    elif dog_detector(proc_tensor):
        print('hello, dog! you look like a ...')
    else:
        print('hmm, neither human nor dog! but it reminds me of ...')
    print(student_model_predict_breed(proc_tensor))

In [22]:
full_pipeline_student('test_images/human.jpg')

hello, human! you vaguely resemble ...
Kerry_blue_terrier


In [23]:
full_pipeline_student('test_images/dog.jpg')

hello, dog!  you look like a ...
Boston_terrier
